<a href="https://colab.research.google.com/github/DiogoMonteiro99/repos/blob/main/casas_boston_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Importando as biblitecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statistics as sts
import warnings
from scipy.stats import probplot,shapiro,spearmanr,pearsonr,kendalltau
from statsmodels import stats
from statsmodels.stats.diagnostic import lilliefors
import statsmodels
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error
from google.cloud import bigquery


warnings.filterwarnings('ignore')

credencial = r'/content/drive/MyDrive/Colab Notebooks/datacienceacadamy-65356b6a3131.json'

casas = bigquery.Client.from_service_account_json(credencial)

query = """SELECT * FROM `datacienceacadamy.Aulas.Casas` """

resultado = casas.query(query)

df = resultado.to_dataframe()

df.head()

**Entendendo as colunas**

RM: É o número médio de cômodos entre os imoveis no bairro.

LSTAT: É a porcentagem de proprietários no bairro considerados de "classe baixa" (proletariado)

PTRATIO: É a razão entre estudantes e professores nas escolas de ensino fundamental e médio no bairro.

**Coluna alvo**

MEDV: Valor médio das casas        

In [ ]:
class Analisar():
    def __init__(self,dados):
        self.df = dados
    def analise(self):
        for i in self.df.columns:
            if self.df[i].dtype == 'int64' or self.df[i].dtype == 'float64':
                print(f'\n Nome da coluna:\n {self.df[i].name}')
                print(f'\n Tipo da coluna:\n {self.df[i].dtype}')
                print(f'\n Tipo da coluna:\n {self.df.corr()}')
                print(f'\n Valor media da coluna:\n%.2f%%' round(self.df[i].mean(),2))
                print(f'\n Quantidade de linhas da coluna:\n {self.df[i].shape}')
                print(f'\n Valores unicos da coluna:\n {self.df[i].isnull().sum()}')
                print(f'\n Valores nulos da coluna:\n {self.df[i].describe()}')
                fig = px.box(self.df[i])
                fig.update_layout(xaxis=dict(showgrid=False),yaxis=dict(showgrid=False),title=f'Boxplot da coluna {i}',template='plotly_dark')
                fig.show()
Analise = Analisar(df)

In [ ]:
Analise.analise()

In [ ]:
fig = px.scatter_matrix(df,dimensions=["RM", "LSTAT", "PTRATIO", "MEDV"])
fig.update_layout(height=800,width=800,title='Correlação',template='plotly_dark')
fig.show()

# **Análise da Normalidade**

Gráfico QQ-Plot

In [ ]:
probplot(df.MEDV,dist='norm',plot=plt)
plt.style.use('dark_background')
plt.title('Normal Q-Q plot')
plt.show()

In [ ]:
df.columns

In [ ]:
fig1 = px.histogram(df,x='MEDV',nbins=50,template='plotly_dark')
fig1.update_layout(title_text='Média dos valores')
fig1.show()

In [ ]:
probplot(df.RM,dist='norm',plot=plt)
plt.style.use('dark_background')
plt.title('Normal Q-Q plot')
plt.show()

In [ ]:
fig2 = px.histogram(df,x='RM',nbins=40,template='plotly_dark')
fig2.update_layout(title_text='Média dos quartos')
fig2.show()

In [ ]:
probplot(df.LSTAT,dist='norm',plot=plt)
plt.style.use('dark_background')
plt.title('Normal Q-Q plot')
plt.show()

In [ ]:
fig3 = px.histogram(df,x='LSTAT', nbins=40,template='plotly_dark')
fig3.update_layout(title_text='Porcentagem de moradores de classe baixa')
fig3.show()

In [ ]:
probplot(df.PTRATIO,dist='norm',plot=plt)
plt.style.use('dark_background')
plt.title('Normal Q-Q plot')
plt.show()

In [ ]:
fig4 = px.histogram(df,x='PTRATIO',nbins=40,template='plotly_dark')
fig4.update_layout(title_text='Razão entre estudante e professores')
fig4.show()

##Teste de Shapiro-Wilk
Shapiro-Wilk pode ser usado no maximo 5000 linhas

Ho = distribuição normal: p > 0.05

Ha = distribuição != normal: p <= 0.05

In [ ]:
shapiro(df.MEDV)

In [ ]:
estatistica,p =shapiro(df.MEDV)
print('Shapiro-Wilk')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {round(p,4)}')

In [ ]:
estatistica,p =shapiro(df.RM)
print('Shapiro-Wilk')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {round(p,4)}')

In [ ]:
estatistica,p =shapiro(df.LSTAT)
print('Shapiro-Wilk')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {round(p,4)}')

In [ ]:
estatistica,p =shapiro(df.PTRATIO)
print('Shapiro-Wilk')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {round(p,2)}')

# **Teste lilliefors (Kolmogorov_Sminov)**
##Lilliefors pode ser usado com mais de 5000 linhas

Ho = distribuição normal: p > 0.05

Ha = distribuição normal: p<= 0.05

In [ ]:
estatistica,p =statsmodels.stats.diagnostic.lilliefors(df.MEDV,dist='norm')
print('Lilliefors')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {round(p,2)}')

In [ ]:
estatistica,p =statsmodels.stats.diagnostic.lilliefors(df.RM,dist='norm')
print('Lilliefors')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {round(p,2)}')

In [ ]:
estatistica,p =statsmodels.stats.diagnostic.lilliefors(df.LSTAT,dist='norm')
print('Lilliefors')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {round(p,2)}')

In [ ]:
estatistica,p =statsmodels.stats.diagnostic.lilliefors(df.PTRATIO,dist='norm')
print('Lilliefors')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {round(p,2)}')

# **Correlação Linear**
##Pearson (distribuição normal)

##Spearman (distribuição não normal)

##Kendall (distribuição não normal com quantidade pequena de amostras)

Ho = Não há correlação linear: p > 0.05

Ha = existe correlação linear: p <= 0.05

**Pearson**

In [ ]:
coef,p = pearsonr(df.MEDV,df.RM)
print('Pearson')
print(f'Coeficiente de correlação: {round(coef,2)}')
print(f'p-valor: {round(p,2)}')

In [ ]:
coef,p = spearmanr(df.MEDV,df.PTRATIO)
print('Spearman')
print(f'Coeficiente de correlação: {round(coef,2)}')
print(f'p-valor: {round(p,2)}')

In [ ]:
coef,p = kendalltau(df.MEDV,df.RM)
print('Kendall')
print(f'Coeficiente de correlação: {round(coef,2)}')
print(f'p-valor: {round(p,2)}')

In [ ]:
correlacoes = df.corr(method='spearman')
correlacoes

In [ ]:
plt.figure()
sns.heatmap(correlacoes,annot=True)
sns.set_theme(style='darkgrid')

# **Regressão Linear Simples**

##Valor médio (MEDV) em função do número de cômodos (RM)

In [ ]:
df.head(2)

In [ ]:
x1 = df.iloc[:,0:1].values
x1

In [ ]:
y = df.iloc[:,3].values
y

In [ ]:
x_treino,x_teste,y_treino,y_teste = train_test_split(x1,y, test_size=0.3, random_state=10,)

In [ ]:
x_treino.shape,y_treino.shape

In [ ]:
x_teste.shape,y_teste.shape

In [ ]:
reg_linear1 = LinearRegression()
reg_linear1.fit(x_treino,y_treino)

Intercepto (coeficiente linear)

In [ ]:
reg_linear1.intercept_

Coeficiente Angular

In [ ]:
reg_linear1.coef_

###**Equação: Valor = -626510.26 + 173771.45 vezes o número cômodos**

Coeficiente de Determinação dados de treino

In [ ]:
reg_linear1.score(x_treino,y_treino)

In [ ]:
reg_linear1.score(x_teste,y_teste)

In [ ]:
previsoes_treino = reg_linear1.predict(x_treino)
previsoes_treino

In [ ]:
plt.scatter(y=y_treino,x=x_treino,color='orange', s=10, alpha=0.9)
x_plot = np.linspace(3,9)
plt.plot(x_plot,x_plot*reg_linear1.coef_ + reg_linear1.intercept_,color='r')
plt.style.use('dark_background')
plt.title('Reta de regressão')
plt.ylabel('Valor médio')
plt.xlabel('Quantidades de cômodos')
plt.show()

In [ ]:
previsoes_teste = reg_linear1.predict(x_teste)
previsoes_teste

In [ ]:
y_teste

In [ ]:
plt.scatter(y=y_teste,x=x_teste, color='orange', s=10, alpha=0.9)
x_plot = np.linspace(3,9)
plt.plot(x_plot,x_plot*reg_linear1.coef_ + reg_linear1.intercept_,color='r')
plt.style.use('dark_background')
plt.title('Reta de regressão')
plt.ylabel('Valor médio')
plt.xlabel('Quantidades de cômodos')
plt.show()

Fazendo previsões para valores distintos

In [ ]:
valor_casa =reg_linear1.predict([[4]])
valor_casa

# **Métricas de Desempenho**

Erro absoluto

In [ ]:
abs(y_teste - previsoes_teste).mean()

Erro médio absoluto

In [ ]:
mean_absolute_error(y_teste,previsoes_teste)

Erro quadrádico médio

In [ ]:
mean_squared_error(y_teste,previsoes_teste)

Raiz do erro quadrático médio

In [ ]:
np.sqrt(mean_squared_error(y_teste,previsoes_teste))

### **Valor médio (MEDV) em função da classe social (LSTAT)**

In [ ]:
df.head(2)

In [ ]:
x2 = df.iloc[:,1:2].values

In [ ]:
y = df.iloc[:,3].values

In [ ]:
x_treino,x_teste,y_treino,y_teste = train_test_split(x2,y,test_size=0.3,random_state=10)


In [ ]:
x_treino.shape,y_treino.shape

In [ ]:
x_teste.shape,y_teste.shape

In [ ]:
reg_linear2 = LinearRegression()
reg_linear2.fit(x_treino,y_treino)

Intercepto (coeficinte linear)

In [ ]:
reg_linear2.intercept_

Coeficiente Angular

In [ ]:
reg_linear2.coef_

### **Equação: Valor = 683187.41 - 17660.54 . LSTAT**

Coeficiente de Determinacão dados de treino

In [ ]:
reg_linear2.score(x_treino,y_treino)

Previsão

In [ ]:
previsoes_treino1 = reg_linear2.predict(x_treino)
previsoes_treino1

In [ ]:
plt.scatter(y=y_treino,x=x_treino, color='orange', s=10, alpha=0.9)
x_plot = np.linspace(0,40)
plt.plot(x_plot,x_plot*reg_linear2.coef_ + reg_linear2.intercept_,color='r')
plt.style.use('dark_background')
plt.title('Reta de regressão')
plt.ylabel('Valor médio')
plt.xlabel('Quantidades de Proprietários de Classe baixa (%)')
plt.show()

In [ ]:
# Prevendo os dados de teste
previsoes_teste = reg_linear2.predict(x_teste)
previsoes_teste

In [ ]:
# Analisando o score
reg_linear2.score(x_teste,y_teste)

In [ ]:
plt.scatter(y=y_teste,x=x_teste, color='orange', s=10, alpha=0.9)
x_plot = np.linspace(2,35)
plt.plot(x_plot,x_plot*reg_linear2.coef_ + reg_linear2.intercept_,color='r')
plt.style.use('dark_background')
plt.title('Reta de regressão')
plt.ylabel('Valor médio')
plt.xlabel('Quantidades de Proprietários de Classe baixa (%)')
plt.show()

In [ ]:
# Fazendo previsões para valores distintos
valor_casa = reg_linear2.predict([[32]])
valor_casa

### Métricas de Desempenho

In [ ]:
abs(y_teste - previsoes_teste).mean()

In [ ]:
# Erro médio absoluto
mean_absolute_error(y_teste, previsoes_teste)